In [ ]:
from Environment import SIERDModel

# Define the problem
problem = {
    'num_vars': 4,
    'names': ['transmission_rate', 'latency_period', 'infection_duration', 'recovery_rate'],
    'bounds': [
               [0.1, 1.0],    # transmission_rate range
               [10, 70],     # latency_period range
               [10, 70],     # infection_duration range
               [0.1, 1.0]]   # recovery_rate range
}

# Generate parameter samples
param_values = saltelli.sample(problem, 128)

# Define the simulation function
def run_sobol_simulation(param_values, width, height, density, policy, num_districts, initial_infected, steps):
    results = []

    for params in param_values:
        transmission_rate, latency_period, infection_duration, recovery_rate = params
        model = SIERDModel(width, height, density, transmission_rate, latency_period, infection_duration, recovery_rate, policy, num_districts, initial_infected)
        
        for _ in range(steps):
            model.step()

        result = model.datacollector.get_model_vars_dataframe()
        results.append(result)
    
    return results

# Simulation parameters
width = 10
height = 10
density = 8
policy = "No Interventions"
num_districts = 5
initial_infected = 50
steps = 500

# Run the simulations
results = run_sobol_simulation(param_values, width, height, density, policy, num_districts, initial_infected, steps)

# Extract output data for Sobol analysis
Y = [result['Infected'].mean() for result in results]

# Perform Sobol sensitivity analysis
Si = sobol.analyze(problem, np.array(Y), print_to_console=True)

# Plot first-order sensitivity indices
plt.figure()
plt.bar(problem['names'], Si['S1'])
plt.xlabel('Parameter')
plt.ylabel('First-order sensitivity index')
plt.title('First-order Sensitivity')
plt.show()

# Plot total-order sensitivity indices
plt.figure()
plt.bar(problem['names'], Si['ST'])
plt.xlabel('Parameter')
plt.ylabel('Total-order sensitivity index')
plt.title('Total-order Sensitivity')
plt.show()


C:\Users\alext\AppData\Local\Temp\ipykernel_582356\876119988.py:15: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 128)


In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

problem = {
    'num_vars': 5,
    'names': ['density', 'transmission_rate', 'latency_period', 'infection_duration', 'recovery_rate'],
    'bounds': [[0.8, 8],
               [0.1, 1.0],
               [10, 70],
               [10, 70],
               [0.1, 1.0]]
}

In [ ]:
param_values = saltelli.sample(problem, 128)

In [ ]:
from mesa.batchrunner import BatchRunner
import pandas as pd
import numpy as np
from Environment import SIERDModel

def run_sobol_simulation(param_values, width, height, policy, num_districts, initial_infected, steps):
    results = []
    
    for params in param_values:
        density, transmission_rate, latency_period, infection_duration, recovery_rate = params
        model = SIERDModel(width, height, density, transmission_rate, latency_period, infection_duration, recovery_rate, policy, num_districts, initial_infected)
        
        for _ in range(steps):
            model.step()
        
        result = model.datacollector.get_model_vars_dataframe().mean()
        results.append(result)
    
    return pd.DataFrame(results)

# set param
width = 10
height = 10
policy = "No Interventions"
num_districts = 5
initial_infected = 50
steps = 500

# run model
results = run_sobol_simulation(param_values, width, height, policy, num_districts, initial_infected, steps)

In [ ]:
Y = results['Infected'].values  # select the output parameters

Si = sobol.analyze(problem, Y, print_to_console=True)

def plot_index(s, params, i, title=''):
    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        plt.figure()

    l = len(indices)

    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    plt.axvline(0, c='k')

# Plot sensitivity indices
for i, title in zip(['1', '2', 'T'], ['First order sensitivity', 'Second order sensitivity', 'Total order sensitivity']):
    plot_index(Si, problem['names'], i, title)
    plt.show()